In [0]:
# ✨ USE YOUR REAL VALUES BELOW

client_id      = ""     # Application (client) ID
tenant_id      = ""      # Directory (tenant) ID
client_secret  = ""          # Client Secret (Value)

storage_account = "azuresupply2605"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
               "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")

spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)

spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
               f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")


In [0]:
dbutils.fs.ls("abfss://fdestination@azuresupply2605.dfs.core.windows.net/")


[FileInfo(path='abfss://fdestination@azuresupply2605.dfs.core.windows.net/Demand_data', name='Demand_data', size=725335, modificationTime=1763719928000),
 FileInfo(path='abfss://fdestination@azuresupply2605.dfs.core.windows.net/ExternalData.csv', name='ExternalData.csv', size=106198, modificationTime=1763718647000),
 FileInfo(path='abfss://fdestination@azuresupply2605.dfs.core.windows.net/feature_engineering_data.csv', name='feature_engineering_data.csv', size=1385807, modificationTime=1763719865000)]

In [0]:
dbutils.fs.mkdirs(f"abfss://bronze@{storage_account}.dfs.core.windows.net/external")
dbutils.fs.mkdirs(f"abfss://bronze@{storage_account}.dfs.core.windows.net/feature")
dbutils.fs.mkdirs(f"abfss://bronze@{storage_account}.dfs.core.windows.net/demand")


True

In [0]:
external_df = spark.read.csv(
    "abfss://fdestination@azuresupply2605.dfs.core.windows.net/ExternalData.csv",
    header=True,
    inferSchema=True
)

external_df.show(5)


+----------+------------------+------------------+------------------+----------------------+
|      DATE|CLOUD_DEMAND_INDEX|        GDP_GROWTH|         INFLATION|COMPETITOR_PRICE_INDEX|
+----------+------------------+------------------+------------------+----------------------+
|2020-01-01|                67| 3.727301838650201| 4.200878600524534|                   120|
|2020-01-02|                77| 4.657382372114083|3.6821822305513994|                   129|
|2020-01-03|                89| 2.910710224528352| 5.363531613888448|                    93|
|2020-01-04|                93|2.6926987037504917|3.7983149076244174|                   115|
|2020-01-05|                88| 4.488592563558635|7.5169760885293915|                    98|
+----------+------------------+------------------+------------------+----------------------+
only showing top 5 rows


In [0]:
feature_df = spark.read.csv(
    "abfss://fdestination@azuresupply2605.dfs.core.windows.net/feature_engineering_data.csv",
    header=True,
    inferSchema=True
)

feature_df.show(5)


+----------+-------+-------+-----------------+----------------+--------+----------+------+----------+------------+-----------+-----------+---------------------+------------------+
|      Date| Region|Service|Daily_Usage_Units|Peak_Usage_Units|VM_Count|Storage_TB|Season|Econ_Index|Downtime_Min|Usage_Lag_1|Usage_Lag_7|Week_Over_Week_Growth|Seasonality_Factor|
+----------+-------+-------+-----------------+----------------+--------+----------+------+----------+------------+-----------+-----------+---------------------+------------------+
|2020-01-01|East US|Compute|            67008|           82228|    6467|         0|Winter|        88|           0|       NULL|       NULL|                 NULL|1.0858207542704796|
|2020-01-02|East US|Compute|           119766|          147059|   14375|         0|Winter|        90|           1|    67008.0|       NULL|                 NULL|1.0670241792350095|
|2020-01-03|East US|Compute|           151845|          164711|   11810|         0|Winter|        94

In [0]:
demand_df = spark.read.csv(
    "abfss://fdestination@azuresupply2605.dfs.core.windows.net/Demand_data",
    header=True,
    inferSchema=True
)

demand_df.show(5)


+-----------------+----------+------------+----------+----------------+-------------+------+-------+----------+--------+
|Daily_Usage_Units|      Date|Downtime_Min|Econ_Index|Peak_Usage_Units|       Region|Season|Service|Storage_TB|VM_Count|
+-----------------+----------+------------+----------+----------------+-------------+------+-------+----------+--------+
|            67008|2020-01-01|           0|        88|           82228|      East US|Winter|Compute|         0|    6467|
|                0|2020-01-01|           1|       100|               0|      East US|Winter|Storage|       461|       0|
|           127251|2020-01-01|           2|       104|          135018|  West Europe|Winter|Compute|         0|    6019|
|                0|2020-01-01|           2|        94|               0|  West Europe|Winter|Storage|      1018|       0|
|            61531|2020-01-01|           0|        91|           74826|Central India|Winter|Compute|         0|    9029|
+-----------------+----------+--

In [0]:
dbutils.fs.mkdirs("abfss://bronze@azuresupply2605.dfs.core.windows.net/external")
dbutils.fs.mkdirs("abfss://bronze@azuresupply2605.dfs.core.windows.net/feature")
dbutils.fs.mkdirs("abfss://bronze@azuresupply2605.dfs.core.windows.net/demand")


True

In [0]:
external_df.write.mode("overwrite").parquet(
    "abfss://bronze@azuresupply2605.dfs.core.windows.net/external/"
)


In [0]:
feature_df.write.mode("overwrite").parquet(
    "abfss://bronze@azuresupply2605.dfs.core.windows.net/feature/"
)


In [0]:
demand_df.write.mode("overwrite").parquet(
    "abfss://bronze@azuresupply2605.dfs.core.windows.net/demand/"
)
